# Part 2 : Enable Logging

### Import all needed packages

In [1]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [2]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [3]:
log_group_name = '/my/amazon/bedrock/logs'

In [4]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' created successfully.


In [5]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [6]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'ac75a4a0-fdef-4edf-86f2-875ac1e4631f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 Sep 2024 21:36:21 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ac75a4a0-fdef-4edf-86f2-875ac1e4631f'},
  'RetryAttempts': 0}}

In [7]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '4a08355d-8b2d-489d-a2d1-fe44d6cdd09d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 Sep 2024 21:36:25 GMT',
   'content-type': 'application/json',
   'content-length': '604',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4a08355d-8b2d-489d-a2d1-fe44d6cdd09d'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::163141031870:role/c99355a2566044l7118059t1w73183247167-LoggingIAMRole-56V62BCmDkAe',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l7118059t1w731832471-loggings3bucket-4ahftntqbbx5',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l7118059t1w731832471-loggings3bucket-4ahftntqbbx5',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [8]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [9]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet that was created by a group of science fiction writers in the early 21st century. The planet is located in a distant galaxy and is known for its unique and exotic features.

One of the most notable features of Foobar is its atmosphere. The planet has a thick atmosphere of methane and nitrogen, which makes it a hostile environment for human life. The surface of the planet is covered in vast deserts, with occasional patches of lush vegetation. The atmosphere is also extremely thin, making it difficult for spacecraft to enter the planet's orbit.

Foobar is home to a wide variety of life forms. The planet's deserts are home to a variety of creatures, including sandworms, scorpions, and lizards. The planet's lush vegetation is home to a variety of plants, including trees and grasses. The planet's oceans are home to a variety of marine life, including fish, whales, and sharks.

The planet's climate is also unique. The planet has a very high temperature, with tem

In [10]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-09-17T21:36:32Z",
    "accountId": "163141031870",
    "identity": {
        "arn": "arn:aws:sts::163141031870:assumed-role/voclabs/user3378214=0"
    },
    "region": "us-west-2",
    "requestId": "7d79770a-f1f5-4a7c-a4e2-c19bc8fb8d3f",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "inputTextTokenCount": 10,
    

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [11]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [12]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
